# Select node and link

In [1]:
import os
import folium
import pandas as pd
import numpy as np

nodes = pd.read_csv('../data/processed_data/node_link/daegu_nodes.csv')
links = pd.read_csv('../data/processed_data/node_link/daegu_links.csv')

In [2]:
nodes = nodes[['NODE_ID','NODE_NAME','latitude','longitude']]

links = links[['LINK_ID', 'F_NODE','T_NODE', 'MAX_SPD', 'LENGTH']]

lon_lim = nodes['longitude'].map(lambda x : x >= 128.578048176449 and x <= 128.618447013494)
lat_lim = nodes['latitude'].map(lambda x : x >= 35.8603352683553 and x <= 35.8784099937842)
selected_nodes = nodes[lon_lim & lat_lim]

nodes_set = set(selected_nodes['NODE_ID'])

valid_source = links['F_NODE'].map(lambda x : x in nodes_set)
valid_target = links['T_NODE'].map(lambda x : x in nodes_set)

selected_links = links[valid_source & valid_target]

In [3]:
selected_links

,LINK_ID,F_NODE,T_NODE,MAX_SPD,LENGTH
0,1500000100,1500000100,1500000200,70,27.956379
1,1500000200,1500000200,1500000100,70,27.886694
5,1500000506,1500000303,1500000600,70,67.406362
8,1500000605,1500000600,1500000303,70,67.231042
10,1500000700,1500000500,1500000400,70,295.186217
...,...,...,...,...,...
8791,1550382600,1550107700,1550107800,50,41.248873
8798,1550383300,1550108000,1550015900,50,305.945975
8799,1550383400,1550015900,1550108000,50,284.441732
8800,1550383500,1550108100,1550015900,50,331.998825


# Delete redundant node

In [5]:
links_set = set(selected_links['F_NODE']).union(set(selected_links['T_NODE']))

selected_nodes = selected_nodes[selected_nodes['NODE_ID'].map(lambda x : x in links_set)]
selected_nodes

,NODE_ID,NODE_NAME,latitude,longitude
0,1500000100,수성교,35.861733,128.608445
1,1500000200,수성교남단교차로,35.861893,128.608207
5,1500000303,신남네거리,35.864666,128.581640
6,1500000400,삼덕네거리,35.863242,128.602626
7,1500000500,봉산육거리,35.863896,128.599459
...,...,...,...,...
2974,1550107600,수성동4가1031,35.861293,128.616696
2975,1550107700,수성동4가1085-19,35.860335,128.616846
2976,1550107800,수성동4가1085-33,35.860352,128.616396
2978,1550108000,범어3동행정복지센터,35.866285,128.617808


# Show graph

In [6]:
map_osm = folium.Map(location=tuple(selected_nodes.head(1)[['latitude','longitude']].iloc[0]), zoom_start=14) 

for _, row in selected_nodes.iterrows():
    location = (row['latitude'], row['longitude'])
    folium.Circle(
        location=location,
        radius=50,
        color='white',
        weight=1,
        fill_opacity=0.6,
        opacity=1,
        fill_color='black',
        fill=True
    ).add_to(map_osm)    
for _, row in selected_links.iterrows():
    src = tuple(selected_nodes[selected_nodes['NODE_ID']==row['F_NODE']][['latitude','longitude']].iloc[0])
    dest = tuple(selected_nodes[selected_nodes['NODE_ID']==row['T_NODE']][['latitude','longitude']].iloc[0])
    folium.PolyLine(
        locations=[src, dest],
        color='red',
        opacity=0.5,
        weight=2
    ).add_to(map_osm)
    
map_osm

# Save

In [7]:
selected_nodes.to_csv('../data/processed_data/node_link/daegu_selected_nodes.csv', index=False)
selected_links.to_csv('../data/processed_data/node_link/daegu_selected_links.csv', index=False) 